In [1]:
import sys
sys.path.append('/mnt/nvme2tb/ffp/code/mlfires/ML_fires_al')

In [205]:
import pandas as pd
from fileutils import find_files
from MLscores import hybridrecall
import json
import os

In [3]:
def loadresults(folder, partern):
    resdfs=[]
    for f in find_files(folder, partern, listtype="list"):
        resdfs+=[pd.read_csv(f)]
    return pd.concat(resdfs)

In [4]:
def get_algo(params):
    pdict=eval(params)
    if 'dropout' in pdict and pdict['dropout'] is None:
        return 'NN'
    elif 'dropout' in pdict and pdict['dropout'] is not None:
        return 'NNd'
    elif 'algo' in pdict:
        return pdict['algo']
    
def get_col_seq(cols):
    colseq=[]
    for al in ['NN ', 'NNd', 'RF','ET', 'XGB']:
        colseq+=sorted([c for c in cols if al in c])
    return colseq

In [208]:
'''Create dataframe with best results from test runs'''

def allbest(folder, partern, allbestfile=None, bestmodfile=None, allmodfile=None):
    # loading results csv files from specific folder with specific pattern
    resdf=loadresults(folder, partern)
    
    # creat column with algorithm name
    resdf['algo']=resdf['params'].apply(lambda p: get_algo(p))
    
    # print info
    print('Total models %d'%(int(len(resdf)/5)))
    print('Algorithms %s'%sorted(list(resdf['algo'].unique())))
    print('Metrics %s'%sorted(list(resdf['opt. metric'].unique())))
    
    # calculate BA for selecting best result
    resdf['BA test'] = hybridrecall(1, 1, resdf['recall 1 test'], resdf['recall 0 test'], 'NH')
    #resdf['BA test'] = resdf['recall 1 test']+resdf['recall 0 test']
    
    #save a list with all test models
    if allmodfile is not None:
        allmod=resdf[['Model ID','algo','opt. metric','test set','training set','BA test', 'recall 1 test', 
                      'recall 0 test', 'TP test', 'FN test', 'TN test', 'FP test', 'params']].copy()
        allmod.rename(columns={'BA test':'BA', 'recall 1 test':'Sensitivity', 'recall 0 test':'Specificity',
                              'TP test':'TP', 'FP test':'FP', 'TN test':'TN', 'FN test':'FN'}, inplace=True)
        allmod.sort_values(by=['algo','opt. metric']).to_csv(allmodfile, index=False)
    
    # select columns
    resdf0=resdf.loc[resdf['test set']=='all set'][['Model ID','algo','opt. metric', 'BA test', 
                                                    'recall 1 test', 'recall 0 test','params']]
    
    # group by max
    maxba=resdf0.groupby(['algo','opt. metric']).max()[['BA test']].reset_index()
    
    # merge with initial results to keep only results with the best BA values per metric/algo
    maxba_tmp=pd.merge(resdf0, maxba, on='BA test', suffixes=(None, "_r"))
    maxbaf=maxba_tmp[[c for c in maxba_tmp.columns if '_r' not in c]].copy()
    maxbaf.rename(columns={'BA test':'BA', 'recall 1 test':'Sens.', 'recall 0 test':'Spec.'}, inplace=True)
    
    # apply naming conventions
    maxbaf['opt. metric']=maxbaf['opt. metric'].str.replace(' test','')
    maxbaf['opt. metric']=maxbaf['opt. metric'].str.replace('BA','sh1')
    maxbaf['opt. metric']=maxbaf['opt. metric'].str.replace('hybrid','rh')
    maxbaf['opt. metric']=maxbaf['opt. metric'].str.replace('NH','sh')
    maxbaf['opt. metric']=maxbaf['opt. metric'].str.replace('f1-score 1','f1')
    maxbaf['algo']=maxbaf['algo'].str.replace('XT','ET')
    
    if bestmodfile is not None:
        maxbaf.to_csv(os.path.join(folder,bestmodfile),index=False) 
    
    # build final dataframe
    reslines=[]
    for om in ['auc', 'f1', 'rh1', 'rh2', 'rh5', 'sh1', 'sh2', 'sh5', 'sh10']:
        resline={}
        resline['metric']=om
        for al in sorted(list(maxbaf['algo'].unique())):
            maxb_row=maxbaf[(maxbaf['algo']==al)&(maxbaf['opt. metric']==om)]
            if not maxb_row.empty:
                resline['%s Sens.'%al]=round(maxb_row.iloc[0]['Sens.'],2)
                resline['%s Spec.'%al]=round(maxb_row.iloc[0]['Spec.'],2)
                resline['%s BA'%al]=round(maxb_row.iloc[0]['BA'],2)
        reslines+=[resline]
    ba_df=pd.DataFrame(reslines)
    ba_df_ord=ba_df[['metric']+get_col_seq(ba_df.columns)]
    if allbestfile is not None:
        ba_df_ord.to_csv(os.path.join(folder,allbestfile),index=False) 
    return ba_df_ord
    

In [209]:
folder='/mnt/nvme2tb/ffp/results/best2/test'
pattern='*_ns_gr*2019*all*'
resdf=allbest(folder, pattern,allbestfile='allbest_gr_2019.csv', 
              bestmodfile='allbest_gr_models_2019.csv', 
              allmodfile='allbesttest_gr_2019.csv')
resdf

Total models 133
Algorithms ['NN', 'NNd', 'RF', 'XGB', 'XT']
Metrics ['BA test', 'NH10 test', 'NH2 test', 'NH5 test', 'auc test', 'f1-score 1 test', 'hybrid1 test', 'hybrid2 test', 'hybrid5 test']


,metric,NN BA,NN Sens.,NN Spec.,NNd BA,NNd Sens.,NNd Spec.,RF BA,RF Sens.,RF Spec.,ET BA,ET Sens.,ET Spec.,XGB BA,XGB Sens.,XGB Spec.
0,auc,1.64,0.82,0.83,1.65,0.83,0.82,1.14,0.15,0.99,1.23,0.25,0.98,1.62,0.77,0.85
1,f1,1.67,0.87,0.80,1.65,0.84,0.81,1.65,0.87,0.78,1.65,0.87,0.78,1.61,0.79,0.83
2,rh1,1.65,0.80,0.85,1.67,0.86,0.81,1.62,0.77,0.85,1.66,0.86,0.80,1.61,0.79,0.83
3,rh2,1.67,0.84,0.83,1.67,0.86,0.81,1.65,0.89,0.76,1.63,0.89,0.74,1.63,0.83,0.80
4,rh5,1.66,0.91,0.74,1.64,0.89,0.75,1.60,0.91,0.69,1.61,0.92,0.69,1.62,0.89,0.73
5,sh1,1.65,0.82,0.83,1.67,0.84,0.82,1.64,0.83,0.81,1.66,0.86,0.79,1.61,0.79,0.83
6,sh2,1.65,0.91,0.75,1.65,0.90,0.75,1.61,0.90,0.71,1.62,0.92,0.70,1.62,0.89,0.73
7,sh5,1.61,0.93,0.68,1.61,0.92,0.69,1.54,0.94,0.59,1.53,0.96,0.57,1.56,0.96,0.60
8,sh10,1.59,0.95,0.65,1.59,0.94,0.65,1.51,0.95,0.56,1.48,0.98,0.51,1.56,0.96,0.60


In [197]:
folder='/mnt/nvme2tb/ffp/results/best2/test'
pattern='*_ns_att*2019*all*'
bestmod, resatt=allbest(folder, pattern)
resatt.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbest_att.csv',index=False)
bestmod.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbest_att_models.csv',index=False)
resatt

Total models 135
Algorithms ['NN', 'NNd', 'RF', 'XGB', 'XT']
Metrics ['BA test', 'NH10 test', 'NH2 test', 'NH5 test', 'auc test', 'f1-score 1 test', 'hybrid1 test', 'hybrid2 test', 'hybrid5 test']


,metric,NN BA,NN Sens.,NN Spec.,NNd BA,NNd Sens.,NNd Spec.,RF BA,RF Sens.,RF Spec.,ET BA,ET Sens.,ET Spec.,XGB BA,XGB Sens.,XGB Spec.
0,auc,1.37,0.92,0.44,1.36,0.92,0.44,1.27,0.35,0.93,1.24,0.35,0.89,1.51,1.0,0.51
1,f1,1.37,0.85,0.52,1.42,0.96,0.46,1.41,1.00,0.41,1.38,1.00,0.38,1.47,1.0,0.47
2,rh1,1.42,0.96,0.46,1.37,0.88,0.49,1.49,1.00,0.49,1.43,1.00,0.43,1.47,1.0,0.47
3,rh2,1.37,0.96,0.41,1.40,0.96,0.43,1.37,1.00,0.37,1.34,1.00,0.34,1.34,1.0,0.34
4,rh5,1.29,0.96,0.33,1.28,0.92,0.36,1.23,1.00,0.23,1.25,1.00,0.25,1.29,1.0,0.29
5,sh1,1.38,0.96,0.41,1.33,0.88,0.45,1.46,1.00,0.46,1.42,1.00,0.42,1.47,1.0,0.47
6,sh2,1.32,0.96,0.35,1.27,1.00,0.27,1.25,1.00,0.25,1.26,1.00,0.26,1.34,1.0,0.34
7,sh5,1.24,0.92,0.32,1.25,0.96,0.29,1.14,1.00,0.14,1.13,1.00,0.13,1.15,1.0,0.15
8,sh10,1.30,1.00,0.30,1.22,1.00,0.22,1.10,1.00,0.10,1.11,1.00,0.11,1.15,1.0,0.15


In [13]:
folder='/mnt/nvme2tb/ffp/results/best2/test'
pattern='*_ns_atster*2019*all*'
bestmod,resatster=allbest(folder, pattern)
resatster.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbest_atster.csv',index=False)
bestmod.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbest_atster_models.csv',index=False)
resatster

Total of 675 results lines
Algorithms ['NN', 'NNd', 'RF', 'XGB', 'XT']
Metrics ['BA test', 'NH10 test', 'NH2 test', 'NH5 test', 'auc test', 'f1-score 1 test', 'hybrid1 test', 'hybrid2 test', 'hybrid5 test']


,metric,NN BA,NN Sens.,NN Spec.,NNd BA,NNd Sens.,NNd Spec.,RF BA,RF Sens.,RF Spec.,ET BA,ET Sens.,ET Spec.,XGB BA,XGB Sens.,XGB Spec.
0,auc,1.55,0.80,0.75,1.58,0.83,0.75,1.01,0.04,0.98,1.05,0.09,0.96,1.52,0.80,0.73
1,f1,1.60,0.90,0.70,1.58,0.88,0.71,1.59,0.92,0.68,1.59,0.94,0.65,1.53,0.82,0.71
2,rh1,1.59,0.89,0.70,1.56,0.86,0.70,1.57,0.84,0.73,1.59,0.93,0.66,1.53,0.82,0.71
3,rh2,1.58,0.90,0.68,1.59,0.89,0.70,1.61,0.96,0.65,1.57,0.96,0.62,1.52,0.86,0.66
4,rh5,1.53,0.94,0.59,1.58,0.95,0.63,1.53,0.98,0.55,1.51,0.97,0.54,1.52,0.95,0.57
5,sh1,1.59,0.89,0.70,1.59,0.91,0.68,1.60,0.94,0.66,1.60,0.95,0.64,1.53,0.82,0.71
6,sh2,1.56,0.92,0.63,1.55,0.96,0.59,1.54,0.97,0.57,1.53,0.97,0.56,1.52,0.95,0.57
7,sh5,1.53,0.96,0.57,1.52,0.96,0.56,1.44,0.99,0.45,1.42,1.00,0.42,1.43,0.99,0.44
8,sh10,1.47,0.96,0.51,1.46,0.98,0.48,1.40,0.99,0.41,1.35,1.00,0.35,1.43,0.99,0.44


In [6]:
folder='/mnt/nvme2tb/ffp/results/best2/test'
pattern='*_ns_paper*2019*all*'
bestmod,resatster=allbest(folder, pattern)
#resatster.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbest_atster.csv',index=False)
#bestmod.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbest_atster_models.csv',index=False)
oldmod2019=resatster.dropna().drop(columns=['NNd BA'])
oldmod2019.rename(columns={cl:cl+' 2019' for cl in [c for c in oldmod2019.columns if 'NN' in c]}, inplace=True)

Total of 15 results lines
Algorithms ['NNd']
Metrics ['NH2 test', 'auc test']


In [7]:
folder='/mnt/nvme2tb/ffp/results/best2/test'
pattern='*_ns_paper*2020*all*'
bestmod,resatster=allbest(folder, pattern)
#resatster.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbest_atster.csv',index=False)
#bestmod.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbest_atster_models.csv',index=False)
oldmod2020=resatster.dropna().drop(columns=['NNd BA'])
oldmod2020.rename(columns={cl:cl+' 2020' for cl in [c for c in oldmod2020.columns if 'NN' in c]}, inplace=True)

Total of 10 results lines
Algorithms ['NNd']
Metrics ['NH2 test', 'auc test']


In [10]:
pd.merge(oldmod2019,oldmod2020,on='metric').to_csv('/mnt/nvme2tb/ffp/results/best2/test/all_best_baseline.csv', index=False)

In [210]:
folder='/mnt/nvme2tb/ffp/results/best2/test'
pattern='*_ns_gr*2020*all*'
resdf=allbest(folder, pattern,allbestfile='allbest_gr_2020.csv', 
              bestmodfile='allbest_gr_models_2020.csv', 
              allmodfile='allbesttest_gr_2020.csv')
resdf

Total models 198
Algorithms ['NN', 'NNd', 'RF', 'XGB', 'XT']
Metrics ['BA test', 'NH10 test', 'NH2 test', 'NH5 test', 'auc test', 'f1-score 1 test', 'hybrid1 test', 'hybrid2 test', 'hybrid5 test']


,metric,NN BA,NN Sens.,NN Spec.,NNd BA,NNd Sens.,NNd Spec.,RF BA,RF Sens.,RF Spec.,ET BA,ET Sens.,ET Spec.,XGB BA,XGB Sens.,XGB Spec.
0,auc,1.70,0.83,0.87,1.73,0.90,0.83,1.53,0.58,0.95,1.57,0.63,0.94,1.70,0.87,0.83
1,f1,1.70,0.84,0.86,1.69,0.83,0.87,1.72,0.89,0.83,1.71,0.85,0.85,1.68,0.81,0.87
2,rh1,1.70,0.80,0.90,1.72,0.86,0.86,1.73,0.85,0.88,1.72,0.85,0.87,1.68,0.81,0.87
3,rh2,1.72,0.86,0.86,1.70,0.85,0.86,1.72,0.90,0.83,1.71,0.88,0.83,1.71,0.89,0.82
4,rh5,1.73,0.90,0.82,1.74,0.89,0.85,1.67,0.93,0.75,1.69,0.94,0.75,1.69,0.92,0.78
5,sh1,1.70,0.85,0.85,1.70,0.81,0.88,1.72,0.83,0.88,1.72,0.85,0.87,1.68,0.81,0.87
6,sh2,1.72,0.90,0.82,1.75,0.93,0.82,1.70,0.94,0.76,1.70,0.95,0.75,1.69,0.92,0.78
7,sh5,1.70,0.94,0.77,1.72,0.93,0.79,1.62,0.97,0.65,1.64,0.98,0.66,1.62,0.96,0.66
8,sh10,1.71,0.94,0.77,1.68,0.95,0.73,1.61,0.98,0.63,1.58,0.98,0.60,1.62,0.96,0.66


In [212]:
folder='/mnt/nvme2tb/ffp/results/best2/test'
pattern='*_ns_*attic*2020*all*'
allbest(folder, pattern,allbestfile='allbest_att_2020.csv')

Total models 165
Algorithms ['NN', 'NNd', 'RF', 'XGB', 'XT']
Metrics ['BA test', 'NH10 test', 'NH2 test', 'NH5 test', 'auc test', 'f1-score 1 test', 'hybrid1 test', 'hybrid2 test', 'hybrid5 test']


,metric,NN BA,NN Sens.,NN Spec.,NNd BA,NNd Sens.,NNd Spec.,RF BA,RF Sens.,RF Spec.,ET BA,ET Sens.,ET Spec.,XGB BA,XGB Sens.,XGB Spec.
0,auc,1.57,0.83,0.74,1.56,0.96,0.60,1.54,0.59,0.95,1.57,0.63,0.94,1.46,0.83,0.63
1,f1,1.55,0.96,0.59,1.55,0.97,0.59,1.44,0.97,0.48,1.51,1.00,0.51,1.54,0.96,0.58
2,rh1,1.57,0.96,0.61,1.56,0.97,0.60,1.57,0.97,0.61,1.49,1.00,0.49,1.54,0.96,0.58
3,rh2,1.54,0.98,0.56,1.55,0.96,0.59,1.45,0.97,0.48,1.50,1.00,0.50,1.48,1.00,0.48
4,rh5,1.46,0.99,0.47,1.51,0.99,0.52,1.31,1.00,0.31,1.34,1.00,0.34,1.38,1.00,0.38
5,sh1,1.53,0.96,0.57,1.56,0.96,0.60,1.50,0.96,0.54,1.49,0.98,0.51,1.54,0.96,0.58
6,sh2,1.46,1.00,0.46,1.47,1.00,0.47,1.31,1.00,0.31,1.39,1.00,0.39,1.38,1.00,0.38
7,sh5,1.41,1.00,0.41,1.39,1.00,0.39,1.20,1.00,0.20,1.21,1.00,0.21,1.20,1.00,0.20
8,sh10,1.44,1.00,0.44,1.36,1.00,0.36,1.16,1.00,0.16,1.17,1.00,0.17,1.20,1.00,0.20


In [211]:
folder='/mnt/nvme2tb/ffp/results/best2/test'
pattern='*_ns*attster*2020*all*'
allbest(folder, pattern,allbestfile='allbest_attster_2020.csv.csv')

Total models 134
Algorithms ['NN', 'NNd', 'RF', 'XGB', 'XT']
Metrics ['BA test', 'NH10 test', 'NH2 test', 'NH5 test', 'auc test', 'f1-score 1 test', 'hybrid1 test', 'hybrid2 test', 'hybrid5 test']


,metric,NN BA,NN Sens.,NN Spec.,NNd BA,NNd Sens.,NNd Spec.,RF BA,RF Sens.,RF Spec.,ET BA,ET Sens.,ET Spec.,XGB BA,XGB Sens.,XGB Spec.
0,auc,1.57,0.78,0.79,1.60,0.78,0.82,1.39,0.41,0.98,1.42,0.45,0.98,1.63,0.93,0.69
1,f1,1.58,0.77,0.81,1.58,0.76,0.82,1.53,0.77,0.75,1.55,0.78,0.77,1.62,0.83,0.79
2,rh1,1.63,0.80,0.83,1.59,0.77,0.82,1.60,0.77,0.83,1.55,0.78,0.77,1.62,0.83,0.79
3,rh2,1.57,0.77,0.79,1.57,0.77,0.81,1.54,0.80,0.75,1.51,0.77,0.74,NaN,NaN,NaN
4,rh5,1.60,0.93,0.67,1.63,0.95,0.68,1.49,0.85,0.64,1.57,0.96,0.62,NaN,NaN,NaN
5,sh1,1.59,0.80,0.79,1.57,0.77,0.80,1.55,0.75,0.80,1.59,0.83,0.75,1.62,0.83,0.79
6,sh2,1.61,0.91,0.71,1.64,0.91,0.73,1.51,0.87,0.64,1.56,0.94,0.61,1.58,0.94,0.64
7,sh5,1.62,0.94,0.68,1.59,0.91,0.68,1.46,0.95,0.51,1.49,0.96,0.54,1.46,0.95,0.51
8,sh10,1.58,0.93,0.65,1.56,0.96,0.60,1.41,0.96,0.45,1.42,0.96,0.46,1.46,0.95,0.51


In [167]:
folder='/mnt/nvme2tb/ffp/results/bestmodels/test'
pattern='*_ns_*2019*all*'
bestmod, res=allbest(folder, pattern, '/mnt/nvme2tb/ffp/results/best2/test/alltestdirty_2019.csv')
res.dropna().to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbestdirty_2019.csv',index=False)
bestmod.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbestdirty_models_2019.csv',index=False)
res.dropna()

Total models 53
Algorithms ['NN', 'NNd']
Metrics ['NH10 test', 'NH2 test', 'NH5 test', 'auc test', 'f1-score 1 test', 'hybrid1 test', 'hybrid2 test', 'hybrid5 test']


,metric,NN BA,NN Sens.,NN Spec.,NNd BA,NNd Sens.,NNd Spec.
0,auc,1.69,0.88,0.81,1.69,0.93,0.76
1,f1,1.69,0.82,0.87,1.71,0.85,0.86
2,rh1,1.71,0.86,0.85,1.71,0.85,0.86
3,rh2,1.70,0.89,0.81,1.70,0.89,0.81
4,rh5,1.69,0.90,0.79,1.70,0.92,0.78
6,sh2,1.71,0.90,0.81,1.70,0.91,0.79
7,sh5,1.69,0.92,0.76,1.67,0.89,0.78
8,sh10,1.69,0.94,0.75,1.70,0.94,0.76


In [75]:
folder='/mnt/nvme2tb/ffp/results/bestmodels/test'
pattern='*_ns_gr*2020*all*'
bestmod, res=allbest(folder, pattern,'/mnt/nvme2tb/ffp/results/best2/test/alltestdirty_2020.csv')
res.dropna().to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbestdirty_2020.csv',index=False)
bestmod.to_csv('/mnt/nvme2tb/ffp/results/best2/test/allbestdirty_models_2020.csv',index=False)
res.dropna()

Total models 47
Algorithms ['NN', 'NNd']
Metrics ['NH10 test', 'NH2 test', 'NH5 test', 'auc test', 'f1-score 1 test', 'hybrid1 test', 'hybrid2 test', 'hybrid5 test']


,metric,NN BA,NN Sens.,NN Spec.,NNd BA,NNd Sens.,NNd Spec.
0,auc,1.76,0.88,0.88,1.77,0.93,0.84
1,f1,1.72,0.82,0.90,1.73,0.83,0.90
2,rh1,1.72,0.82,0.91,1.72,0.82,0.90
3,rh2,1.78,0.91,0.87,1.78,0.94,0.84
4,rh5,1.77,0.94,0.83,1.76,0.92,0.84
6,sh2,1.77,0.94,0.83,1.76,0.91,0.85
7,sh5,1.76,0.92,0.84,1.78,0.95,0.84
8,sh10,1.76,0.93,0.83,1.78,0.94,0.84


In [54]:
folder='/mnt/nvme2tb/ffp/results/bestmodels/test'
pattern='*_ns_*2019*all*'
resdfs=[]
for f in find_files(folder, pattern, listtype="list"):
    resdfs+=[pd.read_csv(f)]
resdf=pd.concat(resdfs)

In [168]:
resdf['algo']=resdf['params'].apply(lambda p: get_algo(p))
resdf['BA test'] = hybridrecall(1, 1, resdf['recall 1 test'], resdf['recall 0 test'], 'NH')
resdf0=resdf.loc[resdf['test set']=='all set'][['algo','opt. metric', 'BA test', 'recall 1 test', 'recall 0 test','params']]

In [169]:
maxba=resdf0.groupby(['algo','opt. metric']).max()[['BA test']].reset_index()

In [170]:
maxba

,algo,opt. metric,BA test
0,NN,BA test,1.376178
1,NN,NH10 test,1.296295
2,NN,NH2 test,1.315259
3,NN,NH5 test,1.241328
4,NN,auc test,1.366361
5,NN,f1-score 1 test,1.367977
6,NN,hybrid1 test,1.424470
7,NN,hybrid2 test,1.372468
8,NN,hybrid5 test,1.293605
9,NNd,BA test,1.333700


In [171]:
maxba_tmp=pd.merge(resdf0, maxba, on='BA test', suffixes=(None, "_r"))
maxbaf=maxba_tmp[[c for c in maxba_tmp.columns if '_r' not in c]].copy()
maxbaf.rename(columns={'BA test':'Bal. Acc', 'recall 1 test':'Sens.', 'recall 0 test':'Spec.'}, inplace=True)
maxbaf

,algo,opt. metric,Bal. Acc,Sens.,Spec.,params
0,NNd,BA test,1.333700,0.884615,0.449085,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
1,NNd,auc test,1.363079,0.923077,0.440002,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
2,NNd,f1-score 1 test,1.418985,0.961538,0.457447,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
3,NNd,hybrid1 test,1.373750,0.884615,0.489135,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
4,NNd,hybrid2 test,1.395754,0.961538,0.434216,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
...,...,...,...,...,...,...
59,XT,hybrid2 test,1.336167,1.000000,0.336167,"{'algo': 'XT', 'bootstrap': True, 'class_weigh..."
60,XT,hybrid5 test,1.245472,1.000000,0.245472,"{'algo': 'XT', 'bootstrap': False, 'class_weig..."
61,XT,NH2 test,1.260413,1.000000,0.260413,"{'algo': 'XT', 'bootstrap': True, 'class_weigh..."
62,XT,NH5 test,1.130924,1.000000,0.130924,"{'algo': 'XT', 'bootstrap': False, 'class_weig..."


In [173]:
maxbaf['opt. metric']=maxbaf['opt. metric'].str.replace(' test','')
maxbaf['opt. metric']=maxbaf['opt. metric'].str.replace('BA','sh1')
maxbaf['opt. metric']=maxbaf['opt. metric'].str.replace('hybrid','rh')
maxbaf['opt. metric']=maxbaf['opt. metric'].str.replace('NH','sh')
maxbaf

,algo,opt. metric,Bal. Acc,Sens.,Spec.,params
0,NNd,sh1,1.333700,0.884615,0.449085,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
1,NNd,auc,1.363079,0.923077,0.440002,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
2,NNd,f1-score 1,1.418985,0.961538,0.457447,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
3,NNd,rh1,1.373750,0.884615,0.489135,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
4,NNd,rh2,1.395754,0.961538,0.434216,"{'ES_mindelta': 0.002, 'ES_monitor': 'val_loss..."
...,...,...,...,...,...,...
59,XT,rh2,1.336167,1.000000,0.336167,"{'algo': 'XT', 'bootstrap': True, 'class_weigh..."
60,XT,rh5,1.245472,1.000000,0.245472,"{'algo': 'XT', 'bootstrap': False, 'class_weig..."
61,XT,sh2,1.260413,1.000000,0.260413,"{'algo': 'XT', 'bootstrap': True, 'class_weigh..."
62,XT,sh5,1.130924,1.000000,0.130924,"{'algo': 'XT', 'bootstrap': False, 'class_weig..."


In [84]:
r=maxbaf[(maxbaf['algo']=='XGB')&(maxbaf['opt. metric']=='auc')].iloc[0]
r.loc['Sens.']

0.9999999961538462

In [85]:
reslines=[]
for om in ['auc', 'rh2', 'rh5', 'sh1', 'sh2', 'sh5', 'sh10',]:
    resline={}
    resline['metric']=om
    for al in sorted(list(maxbaf['algo'].unique())):
        maxb_row=maxbaf[(maxbaf['algo']==al)&(maxbaf['opt. metric']==om)]
        if not maxb_row.empty:
            resline['%s Sens.'%al]=maxb_row.iloc[0]['Sens.']
            resline['%s Spec.'%al]=maxb_row.iloc[0]['Spec.']
    reslines+=[resline]
pd.DataFrame(reslines)
    
    

,metric,NN Sens.,NN Spec.,NNd Sens.,NNd Spec.,RF Sens.,RF Spec.,XGB Sens.,XGB Spec.,XT Sens.,XT Spec.
0,auc,0.923077,0.443284,0.923077,0.440002,0.346154,0.926686,1.0,0.512835,0.346154,0.889106
1,rh2,0.961538,0.410929,0.961538,0.434216,NaN,NaN,NaN,NaN,NaN,NaN
2,rh5,0.961538,0.332066,0.923077,0.361139,NaN,NaN,NaN,NaN,NaN,NaN
3,sh1,0.961538,0.414640,0.884615,0.449085,1.000000,0.461629,1.0,0.473283,1.000000,0.424567
4,sh2,0.961538,0.353721,1.000000,0.265155,NaN,NaN,NaN,NaN,NaN,NaN
5,sh5,0.923077,0.318251,0.961538,0.290734,NaN,NaN,NaN,NaN,NaN,NaN
6,sh10,1.000000,0.296295,1.000000,0.223555,NaN,NaN,NaN,NaN,NaN,NaN
